In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import homework2q1c as hw2

In [3]:
#######################################################################
# Run the stochastic gradient descent algorithm with dropouts
#######################################################################
trainfile = 'train_data.txt'
testfile  = 'test_data.txt'
params = hw2.initialize(trainfile, testfile, eta = 0.001, maxiter = 20, 
                        early_termination = False)

Loaded training data of shape: (1000, 3)<type 'numpy.ndarray'>
Loaded testing data of shape: (1000, 3)<type 'numpy.ndarray'>


In [114]:
model = hw2.sgd(**params)
# Total loss function value in an epoch is the sum of 
# loss functions corresponding to every example

Stochastic Gradient Descent: Running epoch 0
Loss: 0.140981651595                    

Total training loss in this epoch: 595.956189081
Total testing loss in this epoch: 575.684559471
Stochastic Gradient Descent: Running epoch 1
Loss: 1.68655286667                    

Total training loss in this epoch: 318.85249165
Total testing loss in this epoch: 298.593448118
Stochastic Gradient Descent: Running epoch 2
Loss: 1.03353864318                    

Total training loss in this epoch: 235.996757015
Total testing loss in this epoch: 240.899612726
Stochastic Gradient Descent: Running epoch 3
Loss: 0.0457441393006                    

Total training loss in this epoch: 221.283178831
Total testing loss in this epoch: 228.908218321
Stochastic Gradient Descent: Running epoch 4
Loss: 0.03556922832                    

Total training loss in this epoch: 204.240332123
Total testing loss in this epoch: 211.471425
Stochastic Gradient Descent: Running epoch 5
Loss: 0.0202380919094                    

In [115]:
######################################################################
# Validate using Keras model
######################################################################
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [124]:
x_train     = params['train_data'][:,:-1]
y_train     = np.array([hw2.label_to_ytrue(2, x) for x in params['train_data'][:, -1]])
x_test      = params['test_data'][:,:-1]
y_test      = np.array([hw2.label_to_ytrue(2, x) for x in params['test_data'][:, -1]])

keras_model = Sequential()
keras_model.add(Dense(3, activation='relu', use_bias = False, kernel_initializer='random_uniform', input_dim = 2))
keras_model.add(Dense(3, activation='relu', use_bias = False, kernel_initializer='random_uniform'))
keras_model.add(Dropout(1.0/3))
keras_model.add(Dense(2, activation='softmax', use_bias = False, kernel_initializer='random_uniform'))
sgd = SGD(lr=params['eta'])
keras_model.compile(optimizer=sgd, loss='mean_squared_error', metrics=['accuracy'])
keras_model.fit(x_train, y_train, batch_size = 1, epochs=params['maxiter'])
score = keras_model.evaluate(x_test, y_test, batch_size=1)
kw = keras_model.get_weights()

Epoch 1/50
1000/1000 [==============================] - 1s - loss: 0.2500 - acc: 0.2170     
Epoch 2/50
1000/1000 [==============================] - 1s - loss: 0.2500 - acc: 0.2120     
Epoch 3/50
1000/1000 [==============================] - 1s - loss: 0.2500 - acc: 0.2150     
Epoch 4/50
1000/1000 [==============================] - 1s - loss: 0.2500 - acc: 0.2180     
Epoch 5/50
1000/1000 [==============================] - 1s - loss: 0.2500 - acc: 0.2080     
Epoch 6/50
1000/1000 [==============================] - 1s - loss: 0.2500 - acc: 0.2240     
Epoch 7/50
1000/1000 [==============================] - 1s - loss: 0.2500 - acc: 0.2130     
Epoch 8/50
1000/1000 [==============================] - 1s - loss: 0.2500 - acc: 0.2050     
Epoch 9/50
1000/1000 [==============================] - 1s - loss: 0.2500 - acc: 0.2160     
Epoch 10/50
1000/1000 [==============================] - 1s - loss: 0.2500 - acc: 0.2020     
Epoch 11/50
1000/1000 [==============================] - 1s - loss: 0

In [125]:
kw = keras_model.get_weights()

In [118]:
print """
##############################################################
Weights corresponding to best testing accuracy (custom SGD): 
##############################################################"""
print "W1: ", model['w'][0]
print "W2: ", model['w'][1]
print "W3: ", model['w'][2]


##############################################################
Weights corresponding to best testing accuracy (custom SGD): 
##############################################################
W1:  [[-1.94362051  3.93211241]
 [ 6.0522785  -3.14764066]
 [-2.55736844  1.33212314]]
W2:  [[  2.57878635e+00  -1.22965297e+00   1.74857916e+00]
 [ -3.28531420e-01   6.56854888e+00  -7.38426162e-01]
 [  3.88404427e+00  -4.56413107e-03   2.07593749e+00]]
W3:  [[-0.03987199  0.02878348 -0.03466736]
 [ 0.039872   -0.02878348  0.03466737]]


In [126]:
print """
##############################################################
Weights obtained from Keras: 
##############################################################"""
print "Keras W1:", kw[0]
print "Keras W2:", kw[1]
print "Keras W3:", kw[2]


##############################################################
Weights obtained from Keras: 
##############################################################
Keras W1: [[ 0.00622058  0.0010522  -0.00709496]
 [ 0.03349522  0.04137478 -0.02858669]]
Keras W2: [[-0.02256859  0.03819745  0.03867364]
 [-0.01596046 -0.03070686 -0.03310263]
 [ 0.02510785 -0.00401345  0.02000277]]
Keras W3: [[ 0.00108399 -0.00402838]
 [ 0.00430376 -0.02744282]
 [ 0.04630783  0.04273371]]


In [127]:
kwt = []
kwt.append(kw[0].T)
kwt.append(kw[1].T)
kwt.append(kw[2].T)

In [121]:
# None of the units will drop out hence we use a 0 probability for dropout
testing_mask = hw2.generate_masks_for_layers(3, [2, 3, 3], [0, 0, 0])

In [122]:

# Find the f_score and other metrics
precision_recall_fscore = hw2.get_precision_recall_fscore(
    params['shuffle_order_testing'],
    params['test_data'], model['w'],
    params['z'],
    params['a'],
    params['y'],
    testing_mask)
print """
#####################################################################
Precision, Recall and F-score for model output by running custom SGD
#####################################################################
"""
print "Precision: ", precision_recall_fscore[0]
print "Recall: ", precision_recall_fscore[1]
print "F-Score: ", precision_recall_fscore[2]


#####################################################################
Precision, Recall and F-score for model output by running custom SGD
#####################################################################

Precision:  [ 1.          0.99800399]
Recall:  [ 0.998  1.   ]
F-Score:  [ 0.998999  0.999001]


In [128]:

# Find the f_score and other metrics
precision_recall_fscore = hw2.get_precision_recall_fscore(
    params['shuffle_order_testing'],
    params['test_data'],
    kwt,
    params['z'],
    params['a'],
    params['y'], 
    testing_mask)
print """
#####################################################################
Precision, Recall and F-score for model generated by running Keras SGD
#####################################################################
"""
print "Precision: ", precision_recall_fscore[0]
print "Recall: ", precision_recall_fscore[1]
print "F-Score: ", precision_recall_fscore[2]



#####################################################################
Precision, Recall and F-score for model generated by running Keras SGD
#####################################################################

Precision:  [ 0.5  0. ]
Recall:  [ 1.  0.]
F-Score:  [ 0.66666667  0.        ]
